In [11]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.13.0


Checkpoint function

In [12]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

Data load & pre-processing function

In [13]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

Performance function

In [14]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)    
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

Model function

In [15]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

Create model in class version

In [16]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform() #RandomNormal에서 glorot_uniform으로 바꿔준다

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

Create model in function version

In [17]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

Define data & Hyper-parameter

In [18]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

Define model & optimizer & writer

In [19]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_xavier'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

Restore checkpoint & start train or test phrase

In [20]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer("C:/Users/johnk/logs/")
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        #checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.6812, train_loss: 2.10868955, train_accuracy: 0.4297, test_Accuracy: 0.2765
Epoch: [ 0] [    1/  468] time: 0.8727, train_loss: 2.00325584, train_accuracy: 0.5703, test_Accuracy: 0.4530
Epoch: [ 0] [    2/  468] time: 1.0951, train_loss: 1.85564995, train_accuracy: 0.5859, test_Accuracy: 0.5527
Epoch: [ 0] [    3/  468] time: 1.3284, train_loss: 1.67541969, train_accuracy: 0.6797, test_Accuracy: 0.5762
Epoch: [ 0] [    4/  468] time: 1.5788, train_loss: 1.67296374, train_accuracy: 0.5703, test_Accuracy: 0.5951
Epoch: [ 0] [    5/  468] time: 1.7812, train_loss: 1.45015478, train_accuracy: 0.7031, test_Accuracy: 0.6230
Epoch: [ 0] [    6/  468] time: 1.9647, train_loss: 1.43620288, train_accuracy: 0.6641, test_Accuracy: 0.7014
Epoch: [ 0] [    7/  468] time: 2.1652, train_loss: 1.22130668, train_accuracy: 0.7969, test_Accuracy: 0.7557
Epoch: [ 0] [    8/  468] time: 2.3946,

Epoch: [ 0] [   74/  468] time: 16.5038, train_loss: 0.35378116, train_accuracy: 0.8906, test_Accuracy: 0.9107
Epoch: [ 0] [   75/  468] time: 16.7033, train_loss: 0.19348158, train_accuracy: 0.9609, test_Accuracy: 0.9071
Epoch: [ 0] [   76/  468] time: 16.9726, train_loss: 0.33677295, train_accuracy: 0.8906, test_Accuracy: 0.9053
Epoch: [ 0] [   77/  468] time: 17.2409, train_loss: 0.19799398, train_accuracy: 0.9219, test_Accuracy: 0.9058
Epoch: [ 0] [   78/  468] time: 17.5650, train_loss: 0.29108360, train_accuracy: 0.9297, test_Accuracy: 0.9062
Epoch: [ 0] [   79/  468] time: 17.8024, train_loss: 0.25343400, train_accuracy: 0.9062, test_Accuracy: 0.9114
Epoch: [ 0] [   80/  468] time: 18.0736, train_loss: 0.22633241, train_accuracy: 0.9531, test_Accuracy: 0.9129
Epoch: [ 0] [   81/  468] time: 18.3170, train_loss: 0.25592691, train_accuracy: 0.9297, test_Accuracy: 0.9181
Epoch: [ 0] [   82/  468] time: 18.5633, train_loss: 0.25378954, train_accuracy: 0.9219, test_Accuracy: 0.9192
E

Epoch: [ 0] [  148/  468] time: 37.7141, train_loss: 0.18827079, train_accuracy: 0.9375, test_Accuracy: 0.9321
Epoch: [ 0] [  149/  468] time: 37.9644, train_loss: 0.24085148, train_accuracy: 0.9062, test_Accuracy: 0.9343
Epoch: [ 0] [  150/  468] time: 38.2277, train_loss: 0.24530309, train_accuracy: 0.8906, test_Accuracy: 0.9353
Epoch: [ 0] [  151/  468] time: 38.5259, train_loss: 0.27207774, train_accuracy: 0.9062, test_Accuracy: 0.9350
Epoch: [ 0] [  152/  468] time: 38.7712, train_loss: 0.21558458, train_accuracy: 0.9219, test_Accuracy: 0.9347
Epoch: [ 0] [  153/  468] time: 39.0256, train_loss: 0.20103037, train_accuracy: 0.9219, test_Accuracy: 0.9309
Epoch: [ 0] [  154/  468] time: 39.2779, train_loss: 0.26778996, train_accuracy: 0.9219, test_Accuracy: 0.9311
Epoch: [ 0] [  155/  468] time: 39.5222, train_loss: 0.22126940, train_accuracy: 0.9219, test_Accuracy: 0.9305
Epoch: [ 0] [  156/  468] time: 39.7486, train_loss: 0.17237356, train_accuracy: 0.9375, test_Accuracy: 0.9287
E

Epoch: [ 0] [  222/  468] time: 54.7934, train_loss: 0.18233731, train_accuracy: 0.9531, test_Accuracy: 0.9483
Epoch: [ 0] [  223/  468] time: 55.0377, train_loss: 0.31981358, train_accuracy: 0.9062, test_Accuracy: 0.9476
Epoch: [ 0] [  224/  468] time: 55.3050, train_loss: 0.14462239, train_accuracy: 0.9453, test_Accuracy: 0.9495
Epoch: [ 0] [  225/  468] time: 55.5453, train_loss: 0.19807598, train_accuracy: 0.9375, test_Accuracy: 0.9490
Epoch: [ 0] [  226/  468] time: 55.8136, train_loss: 0.20917934, train_accuracy: 0.9531, test_Accuracy: 0.9495
Epoch: [ 0] [  227/  468] time: 56.0899, train_loss: 0.17224203, train_accuracy: 0.9688, test_Accuracy: 0.9498
Epoch: [ 0] [  228/  468] time: 56.3811, train_loss: 0.14703710, train_accuracy: 0.9688, test_Accuracy: 0.9493
Epoch: [ 0] [  229/  468] time: 56.6484, train_loss: 0.16986047, train_accuracy: 0.9688, test_Accuracy: 0.9495
Epoch: [ 0] [  230/  468] time: 56.9037, train_loss: 0.21712609, train_accuracy: 0.9062, test_Accuracy: 0.9481
E

Epoch: [ 0] [  296/  468] time: 73.8184, train_loss: 0.14389257, train_accuracy: 0.9531, test_Accuracy: 0.9573
Epoch: [ 0] [  297/  468] time: 74.1575, train_loss: 0.11868663, train_accuracy: 0.9688, test_Accuracy: 0.9582
Epoch: [ 0] [  298/  468] time: 74.3878, train_loss: 0.12625515, train_accuracy: 0.9688, test_Accuracy: 0.9569
Epoch: [ 0] [  299/  468] time: 74.6342, train_loss: 0.10851024, train_accuracy: 0.9766, test_Accuracy: 0.9573
Epoch: [ 0] [  300/  468] time: 74.9563, train_loss: 0.09865859, train_accuracy: 0.9688, test_Accuracy: 0.9565
Epoch: [ 0] [  301/  468] time: 75.2276, train_loss: 0.13578975, train_accuracy: 0.9609, test_Accuracy: 0.9543
Epoch: [ 0] [  302/  468] time: 75.5029, train_loss: 0.18705148, train_accuracy: 0.9375, test_Accuracy: 0.9560
Epoch: [ 0] [  303/  468] time: 75.7392, train_loss: 0.12774712, train_accuracy: 0.9531, test_Accuracy: 0.9576
Epoch: [ 0] [  304/  468] time: 76.0324, train_loss: 0.11661372, train_accuracy: 0.9531, test_Accuracy: 0.9579
E

Epoch: [ 0] [  370/  468] time: 99.1590, train_loss: 0.31352043, train_accuracy: 0.9531, test_Accuracy: 0.9583
Epoch: [ 0] [  371/  468] time: 99.5110, train_loss: 0.19373584, train_accuracy: 0.9531, test_Accuracy: 0.9595
Epoch: [ 0] [  372/  468] time: 99.8481, train_loss: 0.14136055, train_accuracy: 0.9453, test_Accuracy: 0.9610
Epoch: [ 0] [  373/  468] time: 100.2550, train_loss: 0.13198414, train_accuracy: 0.9453, test_Accuracy: 0.9595
Epoch: [ 0] [  374/  468] time: 100.6320, train_loss: 0.17952096, train_accuracy: 0.9297, test_Accuracy: 0.9590
Epoch: [ 0] [  375/  468] time: 101.0180, train_loss: 0.20268261, train_accuracy: 0.9297, test_Accuracy: 0.9594
Epoch: [ 0] [  376/  468] time: 101.3551, train_loss: 0.17897105, train_accuracy: 0.9453, test_Accuracy: 0.9591
Epoch: [ 0] [  377/  468] time: 101.7730, train_loss: 0.08906741, train_accuracy: 0.9766, test_Accuracy: 0.9594
Epoch: [ 0] [  378/  468] time: 102.1829, train_loss: 0.10241467, train_accuracy: 0.9688, test_Accuracy: 0.

Epoch: [ 0] [  444/  468] time: 126.5547, train_loss: 0.08003482, train_accuracy: 0.9688, test_Accuracy: 0.9614
Epoch: [ 0] [  445/  468] time: 126.9546, train_loss: 0.06724347, train_accuracy: 0.9844, test_Accuracy: 0.9597
Epoch: [ 0] [  446/  468] time: 127.2757, train_loss: 0.09417772, train_accuracy: 0.9688, test_Accuracy: 0.9583
Epoch: [ 0] [  447/  468] time: 127.6437, train_loss: 0.09776327, train_accuracy: 0.9766, test_Accuracy: 0.9570
Epoch: [ 0] [  448/  468] time: 127.9599, train_loss: 0.11262657, train_accuracy: 0.9688, test_Accuracy: 0.9576
Epoch: [ 0] [  449/  468] time: 128.3209, train_loss: 0.08592638, train_accuracy: 0.9844, test_Accuracy: 0.9591
Epoch: [ 0] [  450/  468] time: 128.7009, train_loss: 0.10059439, train_accuracy: 0.9766, test_Accuracy: 0.9607
Epoch: [ 0] [  451/  468] time: 129.0121, train_loss: 0.14664871, train_accuracy: 0.9531, test_Accuracy: 0.9618
Epoch: [ 0] [  452/  468] time: 129.3163, train_loss: 0.17384905, train_accuracy: 0.9609, test_Accuracy:

In [ ]:
Test accuracy = 96%